In [1]:
import os

In [2]:
os.getcwd()

'/Users/kyeongchanlee/projects/bigcontest2024/analysis'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'/Users/kyeongchanlee/projects/bigcontest2024'

In [5]:
from graphrag.get_embedding_model import get_embedding_model
from graphrag.graph_search import retrieve_top_k_stores_by_review_graph_embedding, process_review_node
from llm_response.langgraph_graph_state import GraphState
from llm_response.utils.recomm_get_store_nodes.intent_guide import IntentGuide
from llm_response.utils.recomm_get_store_nodes.t2c import text_to_cypher_for_recomm
from llm_response.utils.recomm_get_store_nodes.top_similar_stores import retrieve_top_similar_stores_pk
from llm_response.utils.recomm_get_store_nodes.utils import convert_markdown_to_html
from llm_response.utils.recomm_get_store_nodes.cypher_result_to_str import get_cypher_result_to_str


import streamlit as st
import re

from prompt.text_to_cypher_for_recomm import EXAMPLES_COMBINED, NEO4J_SCHEMA_RECOMM, TEXT_TO_CYPHER_FOR_RECOMM_TEMPLATE
from utils import get_candidate_str
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

/Users/kyeongchanlee/.pyenv/versions/bigcontest2024/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
embedding_model = get_embedding_model()  # 초기화된 모델을 재사용

In [7]:
from llm_response.get_llm_model import get_llm_model
from graphrag.graph_search import get_neo4j_vector_graph
from config import CONFIG

llm = get_llm_model()
from utils import graphdb_driver
store_retriever_grp_emb = get_neo4j_vector_graph().as_retriever(search_kwargs={"k": CONFIG.store_retriever_rev_emb_k})
state = {
    'query' : '협재해수욕장 근처 70대 부모님이랑 같이 가기 좋은 한식당 추천해줘',
    'intent' : '70대 부모님과 함께 편안하고 쾌적하게 식사를 즐길 수 있는, 깔끔하고 정갈한 한식 메뉴를 제공하는 협재해수욕장 근처 한식당'
}

In [ ]:
llm.get_num_tokens('안녕')

In [8]:
state = text_to_cypher_for_recomm(llm, state)

Text2Cypher for RECOMM------------------------------------------------------------------------------
# cypher : 
MATCH (rg:Region)-[:HAS_STORE]->(s:STORE)-[:HAS_VISIT_KEYWORD]->(v:Visit_with)
WHERE rg.name CONTAINS "협재" AND s.MCT_TYPE IN ['가정식', '일식', '한식'] AND v.keyword CONTAINS "부모님"
RETURN s.pk AS pk, s.MCT_NM AS RestaurantName, s.ADDR AS Address, s.menu AS Menu, v.keyword AS VisitWith
LIMIT 500

# input_tokens count : 3530
----------------------------------------------------------------------------------------------------


In [9]:
candidates_1st, summary, keys = graphdb_driver.execute_query(state['t2c_for_recomm'])

In [10]:
candidates_1st

[]

In [16]:
lack_num = CONFIG.recomm_candidates_num - len(candidates_1st)

In [17]:
from graphrag.retriever import get_neo4j_vector

In [19]:
store_retriever_rev_emb = get_neo4j_vector().as_retriever(search_kwargs={"k": 6})

In [23]:
review_retrieval = store_retriever_rev_emb.invoke(state['intent'])

In [ ]:
retrievalQuery_v3

In [31]:
def retrieve_top_k_reviews(store_pk, query_embedding, driver, k=2):
    """
    특정 STORE 노드에 연결된 리뷰 중 유사한 TOP-K 리뷰를 반환합니다.
    """
    query = """
    MATCH (s:STORE {pk: $store_pk})-[:HAS_REVIEW]->(r:Review)
    WHERE r.textEmbedding IS NOT NULL
    RETURN r.text AS text, gds.similarity.cosine(r.textEmbedding, $query_embedding) AS similarity
    ORDER BY similarity DESC
    LIMIT $k
    """
    with driver.session() as session:
        result = session.run(
            query, store_pk=store_pk, query_embedding=query_embedding, k=k
        )
        return [
            {"text": record["text"], "similarity": record["similarity"]}
            for record in result
        ]

In [32]:
def get_candidate_str(candidates, query_embedding, graphdb_driver, review_k):
    drop_dup = []
    for r in candidates:
        if r.metadata['pk'] not in [d.metadata['pk'] for d in drop_dup]:
            drop_dup.append(r)

    candidates_str = ''
    for d in drop_dup:
        # 가게명
        candidates_str += f"가게명 : {d.metadata['storeName']}\n"
        # pk
        candidates_str += f"pk : {d.metadata['pk']}\n"
        # 리뷰
        reviews = retrieve_top_k_reviews(d.metadata['pk'], query_embedding, graphdb_driver, k=review_k)
        if reviews:
            reviews_lst = [f"{ri}. {review['text'][:100]}".strip() for ri, review in enumerate(reviews, start=1)]
            candidates_str += "리뷰 : \n" + '\n'.join(reviews_lst) + "\n"
        # 평점
        ratings_lst = []
        for platform in ['naver', 'kakao', 'google']:
            if (platform in d.metadata['store_Rating']) and (d.metadata['store_Rating'][platform] is not None):
                pf_rating = d.metadata['store_Rating'][platform]
            else:
                continue
            if platform in d.metadata['reviewCount'] and (d.metadata['reviewCount'][platform] is not None):
                pf_rc = d.metadata['reviewCount'][platform]
            else:
                continue
            ratings_lst.append(f"{platform} {pf_rating}({pf_rc}명)")
        rating_str = ', '.join(ratings_lst)
        candidates_str += f"평점 : {rating_str}\n"
        # 주소
        if 'store_Addr' in d.metadata:
            candidates_str += f"주소 : {d.metadata['store_Addr']}\n"
        # 음식 유형
        if 'store_Type' in d.metadata:
            candidates_str += f"음식 유형 : {d.metadata['store_Type']}\n"
        # 방문 목적 top 3 
        if 'purpose' in d.metadata:
            candidates_str += f"방문 목적 top 3 : {d.metadata['purpose']}\n"
        # 대기 시간 통계
        if 'wait_time' in d.metadata:
            wait_time_str = d.metadata['wait_time'].replace('{', '').replace('}', '').replace('"', '')
            candidates_str += f"대기 시간 통계 : {wait_time_str}\n"
        # 예약 필요 여부
        if 'use_how' in d.metadata:
            use_how_str = d.metadata['use_how'].replace('{', '').replace('}', '').replace('"', '')
            candidates_str += f"예약 필요 여부 통계 : {use_how_str}\n"
        # 메뉴
        if 'menu' in d.metadata:
            candidates_str += f"메뉴 : {d.metadata['menu'][:100]}\n"

        candidates_str += '\n'
    return candidates_str

In [34]:
intent = state['intent']
query_embedding = embedding_model.embed_query(intent)

In [36]:
get_candidate_str(review_retrieval[:lack_num], query_embedding, graphdb_driver, review_k=2)

가게명 : 오라장어
pk : 5525
리뷰 : 
1. 부모님 모시고 식사하기 좋아요. 장어도 튼실하고 맛이 좋네요~
2. 반찬도 다양하고 장어구이도 맛있어요 ! 부모님 모시고 왔는데 친절하시고 다음에도 방문하고 싶네요 ㅎㅎ !!💛
평점 : google 3.5(2명)
주소 : 제주 제주시 오라일동 1076번지 1층
음식 유형 : 가정식
방문 목적 top 3 : 가족모임, 친목, 일상
대기 시간 통계 : 30분 이내:2,10분 이내:2,바로 입장:92
예약 필요 여부 통계 : 예약 없이 이용:73,예약 후 이용:20
메뉴 : 골라먹는 특대사이즈 민물장어(100g:8800, (포장)숯불초벌 특대사이즈 민물장어:55000, (진공포장)특대사이즈 생물민물장어:55000, 골라먹는 갈비살(100g):8800,

가게명 : 우미관
pk : 5894
리뷰 : 
1. 부모님 모시고 갔는데 너무 만족스러운 식사였습니다! 이전하고 처음갔는데 내부 공간도 주차장도 훨씬 쾌적했어요 😆 깔끔하고 정갈하고 든든한 한끼식사로 강추합니다!
2. 한상차림으로 깔끔하게 먹기 좋았어요^^
저는 장어덮밥먹고 부모님은 장어구이 드셨는데
장어구이는 좀 더 바삭하게 구웠으면 좋다 하셨어요
덮밥은 맛있었습니다!!
평점 : naver 4.55(224명)
주소 : 제주 제주시 이도이동 1188-20번지 2층
음식 유형 : 가정식
방문 목적 top 3 : 가족모임, 데이트, 친목
대기 시간 통계 : 10분 이내:1,바로 입장:81
예약 필요 여부 통계 : 예약 없이 이용:17,포장·배달 이용:4,예약 후 이용:62
메뉴 : 장어정식(1인):39000, 장어덮밥(특 250g):36000, 장어덮밥(소 180g):29000, 장어구이(추가/포장):34000

가게명 : 온평바다한그릇
pk : 5663
리뷰 : 
1. 부모님 모시고 맛있게 잘 먹고 갑니다.
해산물도 신선하고 잘먹었습니다❤️
2. 서버 분들꼐서 너무 친절하게 대해주셨네요~ 젊으신 분들 보기가 좋았습니다~^^ 해산물 모둠 하나 시켰는데 너무 신선한 것

In [ ]:
len(candidates_1st)